In [1]:
import os, re, openai, torch
from secret import *
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
    retry_if_exception_type,
)  # for exponential backoff
openai.api_key = OPENAI_API_KEY
@retry(
    retry=retry_if_exception_type(
        (
            openai.error.APIError,
            openai.error.APIConnectionError,
            openai.error.RateLimitError,
            openai.error.ServiceUnavailableError,
            openai.error.Timeout,
        )
    ),
    wait=wait_random_exponential(multiplier=1, max=60),
    stop=stop_after_attempt(10),
)
def chat_completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [14]:
import pandas as pd
adjusted_prompt_df=  pd.read_csv('[NoPK] neo4j-vector-adjusted-prompt.csv')


In [17]:
from aug.features import get_report

In [20]:
print(get_report(adjusted_prompt_df[adjusted_prompt_df['chest pain']==True].iloc[0]))


EXAMINATION:  CHEST PA AND LAT  INDICATION:  History: M with chest pain  TECHNIQUE:  Chest PA and lateral  COMPARISON:    FINDINGS:   The patient is status post median sternotomy and CABG.  Cardiac mediastinal and hilar contours are unchanged with the heart size within normal limits. Minimal atherosclerotic calcifications are noted at the aortic knob. Pulmonary vasculature is normal. Calcified granuloma is seen within the right apex.  Lungs are clear. Pulmonary vasculature is normal. No pleural effusion or pneumothorax is present. Minimal degenerative changes are seen within the thoracic spine.  IMPRESSION:   No acute cardiopulmonary abnormality. LESIONS: Atelectasi. AGE: 73. GENDER: Male.


In [10]:
prompt = """
**Prior Knowledge:**
=========
Question: What are the symptoms associated with atelectasis?
Answer: The symptoms associated with atelectasis may include cough (not prominent), chest pain (not common), breathing difficulty (fast and shallow), low oxygen saturation, pleural effusion (transudate type), cyanosis (late sign), and increased heart rate. However, atelectasis can also be asymptomatic. Fever is not a symptom of atelectasis.
Question: What can cause atelectasis?
Answer: Atelectasis can be caused by various medical conditions, including post-surgical complications, surfactant deficiency, and poor surfactant spreading during inspiration. It can also be caused by blockage of a bronchiole or bronchus, such as by a foreign body, mucus plug, tumor, or compression from the outside. Risk factors for atelectasis include certain types of surgery, muscle relaxation, obesity, high oxygen, lower lung segments, age, chronic obstructive pulmonary disease or asthma, and type of anesthetic.
Question: What are the patient’s symptoms that are relevant for atelectasis?
Answer: The relevant symptoms for atelectasis include cough (not prominent), chest pain (not common), breathing difficulty (fast and shallow), low oxygen saturation, pleural effusion (transudate type), cyanosis (late sign), and increased heart rate.
Question: What are the relevant clinical signs for the etiological diagnosis of atelectasis?
Answer: The relevant clinical signs for the etiological diagnosis of atelectasis may include cough, chest pain (not common), breathing difficulty (fast and shallow), low oxygen saturation, pleural effusion (transudate type), cyanosis (late sign), and increased heart rate. However, it is important to note that atelectasis can also be asymptomatic.
Question: What are the relevant laboratory data for the etiological diagnosis of atelectasis?
Answer: The relevant laboratory data for the etiological diagnosis of atelectasis are not provided in the given information.
Question: What are the relevant clinical characteristics for the etiological diagnosis of atelectasis
Answer: The relevant clinical characteristics for the etiological diagnosis of atelectasis include cough (not prominent), chest pain (not common), breathing difficulty (fast and shallow), low oxygen saturation, pleural effusion (transudate type), cyanosis (late sign), and increased heart rate. It is important to note that atelectasis does not cause fever. The incidence of fever actually decreases as the incidence of atelectasis increases. There is no clinical evidence supporting the speculation that atelectasis causes fever.
Question: What are the patient’s personal relevant history for the etiological diagnosis of atelectasis
Answer: The patient's personal relevant history for the etiological diagnosis of atelectasis includes post-surgical atelectasis as a common cause, as well as pulmonary tuberculosis, smoking, and old age as risk factors. Other factors associated with the development of atelectasis include the presence of chronic obstructive pulmonary disease or asthma, and the type of anesthesia used. The diagnosis of atelectasis is generally confirmed through chest X-ray, which shows small volume linear shadows, usually peripherally or at the lung bases. CT chest or bronchoscopy may be necessary to determine the cause or confirm the absence of proximal obstruction.
=========



**Report:**
=========
INDICATION:  Central venous line placement.  TECHNIQUE:  Frontal chest radiograph.  COMPARISON:  Chest radiograph 12:42 today.  FINDINGS:   A right subclavian catheter has been placed in the interim. The catheter terminates at the confluence of the brachiocephalic vein and superior vena cava and if indicated could be advanced 3.7 cm for termination within the low SVC.  There is no pleural effusion or pneumothorax. The cardiac silhouette remains mildly enlarged. There is no focal airspace consolidation worrisome for pneumonia.  High density material is again seen in the paritally imaged colon in the left abdomen. Cholecystectomy clips are noted. There are carotid calcifications left greater than right. LESIONS: Enlarged cardiac silhouett. AGE: 69. GENDER: Female.
=========

Base on the prior knowledge and the report above, please predict whether the patient has following symptoms/clinical signs/laboratory data/clinical characteristics/clinical history? (Note: When it's not showing in the report, it doesn't mean the patient doesn't have it. Please speculate the result.)
1. anesthesia.
2. asthma.
3. asymptomatic.
4. breathing difficulty.
5. chest pain.
6. chronic obstructive pulmonary disease.
7. cough.
8. cyanosis.
9. fever.
10. pleural effusion.
11. pulmonary tuberculosis.
12. small volume linear shadows.
13. smoking.

Please return the answer ({True/False}/False only) with reasons followed by as the example below shown. 

1. anesthesia: {True/False}, because...
2. asthma: {True/False}, because...
3. asymptomatic: {True/False}, because...
4. breathing difficulty: {True/False}, because...
5. chest pain: {True/False}, because...
6. chronic obstructive pulmonary disease: {True/False}, because...
7. cough: {True/False}, because...
8. cyanosis: {True/False}, because...
9. fever: {True/False}, because...
10. pleural effusion: {True/False}, because...
11. pulmonary tuberculosis: {True/False}, because...
12. small volume linear shadows: {True/False}, because...
13. smoking: {True/False}, because...

"""

In [63]:
prompt_with_chest_pain = """
**Prior Knowledge:**
=========
Question: What are the symptoms associated with atelectasis?
Answer: The symptoms associated with atelectasis may include cough (not prominent), chest pain (not common), breathing difficulty (fast and shallow), low oxygen saturation, pleural effusion (transudate type), cyanosis (late sign), and increased heart rate. However, atelectasis can also be asymptomatic. Fever is not a symptom of atelectasis.
Question: What can cause atelectasis?
Answer: Atelectasis can be caused by various medical conditions, including post-surgical complications, surfactant deficiency, and poor surfactant spreading during inspiration. It can also be caused by blockage of a bronchiole or bronchus, such as by a foreign body, mucus plug, tumor, or compression from the outside. Risk factors for atelectasis include certain types of surgery, muscle relaxation, obesity, high oxygen, lower lung segments, age, chronic obstructive pulmonary disease or asthma, and type of anesthetic.
Question: What are the patient’s symptoms that are relevant for atelectasis?
Answer: The relevant symptoms for atelectasis include cough (not prominent), chest pain (not common), breathing difficulty (fast and shallow), low oxygen saturation, pleural effusion (transudate type), cyanosis (late sign), and increased heart rate.
Question: What are the relevant clinical signs for the etiological diagnosis of atelectasis?
Answer: The relevant clinical signs for the etiological diagnosis of atelectasis may include cough, chest pain (not common), breathing difficulty (fast and shallow), low oxygen saturation, pleural effusion (transudate type), cyanosis (late sign), and increased heart rate. However, it is important to note that atelectasis can also be asymptomatic.
Question: What are the relevant laboratory data for the etiological diagnosis of atelectasis?
Answer: The relevant laboratory data for the etiological diagnosis of atelectasis are not provided in the given information.
Question: What are the relevant clinical characteristics for the etiological diagnosis of atelectasis
Answer: The relevant clinical characteristics for the etiological diagnosis of atelectasis include cough (not prominent), chest pain (not common), breathing difficulty (fast and shallow), low oxygen saturation, pleural effusion (transudate type), cyanosis (late sign), and increased heart rate. It is important to note that atelectasis does not cause fever. The incidence of fever actually decreases as the incidence of atelectasis increases. There is no clinical evidence supporting the speculation that atelectasis causes fever.
Question: What are the patient’s personal relevant history for the etiological diagnosis of atelectasis
Answer: The patient's personal relevant history for the etiological diagnosis of atelectasis includes post-surgical atelectasis as a common cause, as well as pulmonary tuberculosis, smoking, and old age as risk factors. Other factors associated with the development of atelectasis include the presence of chronic obstructive pulmonary disease or asthma, and the type of anesthesia used. The diagnosis of atelectasis is generally confirmed through chest X-ray, which shows small volume linear shadows, usually peripherally or at the lung bases. CT chest or bronchoscopy may be necessary to determine the cause or confirm the absence of proximal obstruction.
=========



**Report:**
=========
EXAMINATION:  CHEST PA AND LAT  INDICATION:  History: M with chest pain  TECHNIQUE:  Chest PA and lateral  COMPARISON:    FINDINGS:   The patient is status post median sternotomy and CABG.  Cardiac mediastinal and hilar contours are unchanged with the heart size within normal limits. Minimal atherosclerotic calcifications are noted at the aortic knob. Pulmonary vasculature is normal. Calcified granuloma is seen within the right apex.  Lungs are clear. Pulmonary vasculature is normal. No pleural effusion or pneumothorax is present. Minimal degenerative changes are seen within the thoracic spine.  IMPRESSION:   No acute cardiopulmonary abnormality. LESIONS: Atelectasis. AGE: 73. GENDER: Male.
=========

Please predict whether the patient has following symptoms/clinical signs/laboratory data/clinical characteristics/clinical history? (Note: When a symptom is not evident in the report, it does not necessarily imply the absence of the condition in the patient. Please consider and speculate on the potential results.)

(Note: do not use the report)

1. anesthesia.
2. asthma.
3. asymptomatic.
4. breathing difficulty.
5. chest pain.
6. chronic obstructive pulmonary disease.
7. cough.
8. cyanosis.
9. fever.
10. pleural effusion.
11. pulmonary tuberculosis.
12. small volume linear shadows.
13. smoking.

Please return the answer ({True/False}/False only). Explain your answer.
"""

In [64]:
res = chat_completion_with_backoff(
        model="gpt-3.5-turbo",
        # model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful clinical expert."},
            # {"role": "system", "content": "You are an experienced radiologist. Use a keyword-based report to answer the following questions."},
            {"role": "user", "content": prompt_with_chest_pain},
        ],
        temperature=0,
        n=1,
    )

In [65]:
print(prompt_with_chest_pain)


**Prior Knowledge:**
Question: What are the symptoms associated with atelectasis?
Answer: The symptoms associated with atelectasis may include cough (not prominent), chest pain (not common), breathing difficulty (fast and shallow), low oxygen saturation, pleural effusion (transudate type), cyanosis (late sign), and increased heart rate. However, atelectasis can also be asymptomatic. Fever is not a symptom of atelectasis.
Question: What can cause atelectasis?
Answer: Atelectasis can be caused by various medical conditions, including post-surgical complications, surfactant deficiency, and poor surfactant spreading during inspiration. It can also be caused by blockage of a bronchiole or bronchus, such as by a foreign body, mucus plug, tumor, or compression from the outside. Risk factors for atelectasis include certain types of surgery, muscle relaxation, obesity, high oxygen, lower lung segments, age, chronic obstructive pulmonary disease or asthma, and type of anesthetic.
Question: What

In [67]:
print(res['choices'][0]['message']['content'])

1. Anesthesia - False. The report does not mention the use of anesthesia.
2. Asthma - False. The report does not mention the presence of asthma.
3. Asymptomatic - False. The patient has reported chest pain, which is a symptom.
4. Breathing difficulty - False. The report does not mention any breathing difficulties.
5. Chest pain - True. The patient has reported chest pain.
6. Chronic obstructive pulmonary disease - False. The report does not mention the presence of chronic obstructive pulmonary disease.
7. Cough - False. The report does not mention any cough.
8. Cyanosis - False. The report does not mention any signs of cyanosis.
9. Fever - False. The report does not mention any fever.
10. Pleural effusion - False. The report specifically states that no pleural effusion is present.
11. Pulmonary tuberculosis - False. The report does not mention the presence of pulmonary tuberculosis.
12. Small volume linear shadows - False. The report does not mention the presence of small volume linear

In [69]:
text= """"
1. Anesthesia - False. The report does not mention the use of anesthesia.
2. Asthma - False. The report does not mention the presence of asthma.
3. Asymptomatic - False. The patient has reported chest pain, which is a symptom.
4. Breathing difficulty - False. The report does not mention any breathing difficulties.
5. Chest pain - True. The patient has reported chest pain.
6. Chronic obstructive pulmonary disease - False. The report does not mention the presence of chronic obstructive pulmonary disease.
7. Cough - False. The report does not mention any cough.
8. Cyanosis - False. The report does not mention any signs of cyanosis.
9. Fever - False. The report does not mention any fever.
10. Pleural effusion - False. The report specifically states that no pleural effusion is present.
11. Pulmonary tuberculosis - False. The report does not mention the presence of pulmonary tuberculosis.
12. Small volume linear shadows - False. The report does not mention the presence of small volume linear shadows.
13. Smoking - False. The report does not mention any history of smoking.
"""

In [83]:
ans = text.strip().split("\n")[1][3:]
boolean_ans = ans[ans.index("-")+2:ans.index(".")] 
reason = ans[ans.index(".")+2:]


In [90]:
text.strip().split("\n")[0][3:]

''

In [91]:
"fsdaf".strip().lower()

'fsdaf'

In [36]:
print(res['choices'][0]['message']['content'])

1. anesthesia: False, because the report does not mention the use of anesthesia.
2. asthma: False, because the report does not mention any history or symptoms of asthma.
3. asymptomatic: True, because the report does not mention any symptoms associated with atelectasis such as cough, chest pain, or breathing difficulty.
4. breathing difficulty: False, because the report does not mention any breathing difficulty.
5. chest pain: True, because the report mentions that the patient has a history of chest pain.
6. chronic obstructive pulmonary disease: False, because the report does not mention any history or symptoms of chronic obstructive pulmonary disease.
7. cough: False, because the report does not mention any cough.
8. cyanosis: False, because the report does not mention any signs of cyanosis.
9. fever: False, because the report does not mention any fever.
10. pleural effusion: False, because the report states "No pleural effusion or pneumothorax is present."
11. pulmonary tuberculosis

In [9]:
print(res['choices'][0]['message']['content'])

1. anesthesia: False, because the report does not mention the patient undergoing any recent anesthesia.
2. asthma: False, because the report does not mention the patient having asthma.
3. asymptomatic: True, because the report does not mention any symptoms that the patient is experiencing.
4. breathing difficulty: False, because the report does not mention the patient having difficulty breathing.
5. chest pain: False, because the report does not mention the patient experiencing chest pain.
6. chronic obstructive pulmonary disease: False, because the report does not mention the patient having chronic obstructive pulmonary disease.
7. cough: False, because the report does not mention the patient having a cough.
8. cyanosis: False, because the report does not mention the patient showing signs of cyanosis.
9. fever: False, because the report does not mention the patient having a fever.
10. pleural effusion: False, because the report specifically states that there is no pleural effusion.
11

In [13]:
print(res['choices'][0]['message']['content'])

1. anesthesia: False, because the report does not mention the use of anesthesia.
2. asthma: False, because the report does not mention any history or symptoms of asthma.
3. asymptomatic: True, because the report does not mention any symptoms that are typically associated with atelectasis such as cough, chest pain, or breathing difficulty.
4. breathing difficulty: False, because the report does not mention any signs of breathing difficulty.
5. chest pain: False, because the report does not mention any signs of chest pain.
6. chronic obstructive pulmonary disease: False, because the report does not mention any history or symptoms of chronic obstructive pulmonary disease.
7. cough: False, because the report does not mention any signs of cough.
8. cyanosis: False, because the report does not mention any signs of cyanosis.
9. fever: False, because the report does not mention any signs of fever.
10. pleural effusion: False, because the report specifically states that there is no pleural effu

In [25]:
print(res['choices'][0]['message']['content'])

1. anesthesia: False, because the report does not mention the use of anesthesia.
2. asthma: False, because the report does not mention any history or symptoms of asthma.
3. asymptomatic: True, because the report does not mention any symptoms associated with atelectasis such as cough, chest pain, or breathing difficulty.
4. breathing difficulty: False, because the report does not mention any breathing difficulty.
5. chest pain: True, because the report mentions that the patient has a history of chest pain.
6. chronic obstructive pulmonary disease: False, because the report does not mention any history or symptoms of chronic obstructive pulmonary disease.
7. cough: False, because the report does not mention any cough.
8. cyanosis: False, because the report does not mention any signs of cyanosis.
9. fever: False, because the report does not mention any fever.
10. pleural effusion: False, because the report states "No pleural effusion or pneumothorax is present."
11. pulmonary tuberculosis

In [28]:
print(res['choices'][0]['message']['content'])

1. anesthesia: False, because the report does not mention the patient undergoing any recent anesthesia.
2. asthma: False, because the report does not mention the patient having asthma.
3. asymptomatic: True, because the report does not mention any symptoms associated with atelectasis such as cough, chest pain, or breathing difficulty.
4. breathing difficulty: False, because the report does not mention the patient having breathing difficulty.
5. chest pain: True, because the report mentions that the patient has a history of chest pain.
6. chronic obstructive pulmonary disease: False, because the report does not mention the patient having chronic obstructive pulmonary disease.
7. cough: False, because the report does not mention the patient having a cough.
8. cyanosis: False, because the report does not mention the patient having cyanosis.
9. fever: False, because the report does not mention the patient having a fever.
10. pleural effusion: False, because the report states "No pleural ef

In [33]:
print(res['choices'][0]['message']['content'])

1. anesthesia: False, because the report does not mention the patient undergoing any recent procedures that would require anesthesia.
2. asthma: False, because the report does not mention any history or symptoms of asthma.
3. asymptomatic: True, because the report does not mention any symptoms associated with atelectasis such as cough, chest pain, or breathing difficulty.
4. breathing difficulty: False, because the report does not mention the patient experiencing any breathing difficulty.
5. chest pain: True, because the report mentions that the patient has a history of chest pain.
6. chronic obstructive pulmonary disease: False, because the report does not mention any history or symptoms of chronic obstructive pulmonary disease.
7. cough: False, because the report does not mention the patient experiencing any cough.
8. cyanosis: False, because the report does not mention any signs of cyanosis.
9. fever: False, because the report does not mention the patient having a fever.
10. pleural

In [38]:
print(res['choices'][0]['message']['content'])

1. anesthesia: False, because there is no mention of anesthesia in the report.
2. asthma: False, because there is no mention of asthma in the report.
3. asymptomatic: True, because there is no mention of any symptoms in the report.
4. breathing difficulty: False, because there is no mention of breathing difficulty in the report.
5. chest pain: True, because the report mentions that the patient has chest pain.
6. chronic obstructive pulmonary disease: False, because there is no mention of chronic obstructive pulmonary disease in the report.
7. cough: False, because there is no mention of cough in the report.
8. cyanosis: False, because there is no mention of cyanosis in the report.
9. fever: False, because there is no mention of fever in the report.
10. pleural effusion: False, because there is no mention of pleural effusion in the report.
11. pulmonary tuberculosis: False, because there is no mention of pulmonary tuberculosis in the report.
12. small volume linear shadows: True, becaus